In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
current_inventory = {'1': 300,
                    '2': 300,
                    '3': 100,
                    '4': 100,
                    '5': 50,
                    '6': 250,
                    '7': 10,
                    '8': 0,
                    '9': 1}

In [ ]:
num_stores = 9
current_day = 0

In [ ]:
daily_demands = {'1': [(83,5.8), (15,7), (20,4), (25,6), (10,6), (83,5.8), (20,4)],
                '2': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '3': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '4': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '5': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '6': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '7': [(20,5), (15,7), (20,4), (25,6), (10,6), (8,5), (20,4)],
                '8': [(20,5), (15,7), (20,4), (25,6), (10,6), (6,2), (20,4)],
                '9': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)]}

In [ ]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [ ]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [ ]:
convolution('1', current_day)

In [ ]:
ratio = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio

In [ ]:
current_day = 0

In [ ]:
def update_inventory(i,current_day):
    todays_sales = {i: int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])),
                    }
    current_inventory[i] = max(0, current_inventory[i] - todays_sales[i])
    
    return current_inventory   

In [ ]:
def shipper_candidates(current_day):
    shipper_cand = []
   
    
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean + (ratio*st_dev)
        
        if critical_point < update_inventory(str(i),current_day)[i]:
            shipper_cand.append(i)
    return shipper_cand

In [ ]:
def receiving_candidates(current_day):
    receiving_cand = []
    for i in current_inventory.keys():
        mean, st_dev = convolution(i, current_day)
        critical_point = mean - (ratio*st_dev)
      
        if critical_point > current_inventory[i]:
            receiving_cand.append(i)
    return receiving_cand

In [ ]:

weekly_df = pd.DataFrame(columns=["day", "shipper", "receiver", "transshipment_amount"])
current_day = 0
a=5
for i in range(a+1):
    if current_day <= 5:
        shipping_list=shipper_candidates(current_day)
        receiving_list=receiving_candidates(current_day)
        daily_df = pd.DataFrame(columns=["shipper", "receiver", "transshipment_amount", "transshipment_cost"])
        for i in shipping_list:
            mean,stdev=convolution(i,current_day)
            for k in receiving_list:
                mean2,stdev2=convolution(k,current_day)
                best_x = 0
                min_cost = float('inf')
                inv=current_inventory[i]
                inv2=current_inventory[k]
                hold_d = dict()
                x_values=np.arange(0,inv,5)
                for x in x_values:
                    hold_d[str(x)] = 0
                    sum_cost=0
                    for number in range(100):
                        exp_shipping=np.random.normal(mean,stdev)
                        exp_receiver=np.random.normal(mean2,stdev2)
                        cost = (costs['stockout'] * max(0, exp_receiver - (current_inventory[k] + x))) + \
                       (costs['stockout'] * max(0, exp_shipping - (current_inventory[i] - x))) + \
                       (costs['transshipment'] * x)
                        sum_cost+=cost
                        
                    expected_cost=sum_cost/100
                    hold_d[str(x)] = expected_cost
                
                best_x, min_cost = min(hold_d.items(), key=lambda x: x[1])
                daily_df = pd.concat([daily_df, pd.DataFrame([{
                "shipper": i,
                "receiver": k,
                "transshipment_amount": best_x,
                "transshipment_cost": min_cost
        }])], ignore_index=True)
        print(daily_df)
        
        while not daily_df.empty:
            
            min_idx = daily_df["transshipment_cost"].idxmin()
            min_cost_row = daily_df.loc[min_idx]
            deleted_shipper = min_cost_row["shipper"]
            deleted_receiver = min_cost_row["receiver"]
            transhipment_amount = daily_df.loc[min_idx, "transshipment_amount"]
            print(deleted_shipper)
            print(deleted_receiver)
            print(transhipment_amount)
            current_inventory[str(deleted_shipper)] = current_inventory[str(deleted_shipper)] - int(transhipment_amount)
            current_inventory[str(deleted_receiver)] = current_inventory[str(deleted_receiver)] + int(transhipment_amount)
            
            weekly_df = pd.concat([weekly_df, pd.DataFrame([{
                "day": current_day,
                "shipper": deleted_shipper,
                "receiver": deleted_receiver,
                "transshipment_amount": transhipment_amount
    
        }])], ignore_index=True)
        
            daily_df = daily_df[(daily_df["shipper"] != deleted_shipper) & (daily_df["receiver"] != deleted_receiver)]
            print(daily_df)
        print("Hnagi gün:",current_day)    
        current_day = current_day + 1   
    print(weekly_df),
    print(current_inventory)
if current_day == 6:
    for i in current_inventory.keys():
            
        mean,stdev=daily_demands[i][current_day]
        current_inventory[i]=max(0, current_inventory[i]-int(np.random.normal(mean,stdev)))
print(current_inventory)          